In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import row_number, to_timestamp, year, month, dayofmonth, col

In [2]:
spark = (
    SparkSession.builder.appName("Transaction Processor")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

In [3]:
df = spark.read.csv(
    "hdfs://hadoop:9000/data/transactions/raw/HI-Small_Trans.csv",
    header=True,
    inferSchema=True,
)
df = df.withColumnRenamed("Account2", "From Account") \
       .withColumnRenamed("Account4", "To Account")
df.printSchema()
df.show(5)

root
 |-- Timestamp: string (nullable = true)
 |-- From Bank: integer (nullable = true)
 |-- From Account: string (nullable = true)
 |-- To Bank: integer (nullable = true)
 |-- To Account: string (nullable = true)
 |-- Amount Received: double (nullable = true)
 |-- Receiving Currency: string (nullable = true)
 |-- Amount Paid: double (nullable = true)
 |-- Payment Currency: string (nullable = true)
 |-- Payment Format: string (nullable = true)
 |-- Is Laundering: integer (nullable = true)

+----------------+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+
|       Timestamp|From Bank|From Account|To Bank|To Account|Amount Received|Receiving Currency|Amount Paid|Payment Currency|Payment Format|Is Laundering|
+----------------+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+
|2022/09/01 00:20|       10|   8000EBD30|  

In [4]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, to_timestamp

df = df.withColumn("ts", to_timestamp("Timestamp", "yyyy/MM/dd HH:mm"))
window_spec = Window.orderBy("ts", "From Bank", "From Account", "To Bank", "To Account")
df = df.withColumn("txn_id", row_number().over(window_spec))

df.show(5)

+----------------+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+-------------------+------+
|       Timestamp|From Bank|From Account|To Bank|To Account|Amount Received|Receiving Currency|Amount Paid|Payment Currency|Payment Format|Is Laundering|                 ts|txn_id|
+----------------+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+-------------------+------+
|2022/09/01 00:00|        1|   800057A10|     12| 800473570|        1433.68|         US Dollar|    1433.68|       US Dollar|        Cheque|            0|2022-09-01 00:00:00|     1|
|2022/09/01 00:00|        1|   80005C0A0|   1588| 8003AC470|         427.27|         US Dollar|     427.27|       US Dollar|           ACH|            0|2022-09-01 00:00:00|     2|
|2022/09/01 00:00|        1|   80005E740|   1362| 800256180|         3698.2|         US Dollar|

In [5]:
df.groupBy("Is Laundering").count().show()

+-------------+-------+
|Is Laundering|  count|
+-------------+-------+
|            1|   5177|
|            0|5073168|
+-------------+-------+



In [6]:
df_sus = df.filter(df["Is Laundering"] == 1)

In [7]:
from pyspark.sql.functions import col, lit

from_parties = df_sus.select(
    col("From Bank").alias("Bank"),
    col("From Account").alias("Account")
)

to_parties = df_sus.select(
    col("To Bank").alias("Bank"),
    col("To Account").alias("Account")
)

# Union lại → distinct để lấy tập nghi ngờ
suspect_parties = from_parties.union(to_parties).distinct()

In [8]:
df_from_match = df.join(
    suspect_parties,
    (df["From Bank"] == suspect_parties["Bank"]) &
    (df["From Account"] == suspect_parties["Account"]),
    how="inner"
)

In [9]:
df_to_match = df.join(
    suspect_parties,
    (df["To Bank"] == suspect_parties["Bank"]) &
    (df["To Account"] == suspect_parties["Account"]),
    how="inner"
)

In [10]:
df_related = df_from_match.union(df_to_match).dropDuplicates(["txn_id"])

In [11]:
print(f"📌 Số dòng liên quan đến giao dịch nghi ngờ: {df_related.count()}")
df_related.show(10, truncate=False)

📌 Số dòng liên quan đến giao dịch nghi ngờ: 616596
+----------------+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+-------------------+------+------+---------+
|Timestamp       |From Bank|From Account|To Bank|To Account|Amount Received|Receiving Currency|Amount Paid|Payment Currency|Payment Format|Is Laundering|ts                 |txn_id|Bank  |Account  |
+----------------+---------+------------+-------+----------+---------------+------------------+-----------+----------------+--------------+-------------+-------------------+------+------+---------+
|2022/09/01 00:00|1        |8001364A0   |1      |8001364A0 |10.3           |US Dollar         |10.3       |US Dollar       |Reinvestment  |0            |2022-09-01 00:00:00|6     |1     |8001364A0|
|2022/09/01 00:00|1        |8001BA930   |1411   |8019F58F0 |3690.43        |US Dollar         |3690.43    |US Dollar       |ACH           |0            |2022

In [12]:
df_related.groupBy("Is Laundering").count().show()

+-------------+------+
|Is Laundering| count|
+-------------+------+
|            1|  5177|
|            0|611419|
+-------------+------+



In [13]:
df_related = df_related.withColumn("year", year("ts")) \
       .withColumn("month", month("ts")) \
       .withColumn("day", dayofmonth("ts"))

window_spec = Window.partitionBy("year", "month", "day") \
                    .orderBy("ts", "txn_id")

df_related = df_related.withColumn("txn_id", row_number().over(window_spec))

In [15]:
print(df_related.rdd.getNumPartitions())


4


In [16]:
df_related.write.partitionBy("year", "month", "day") \
  .mode("overwrite") \
  .parquet("hdfs://hadoop:9000/data/transactions/partitioned")

In [17]:
spark.stop()